In [1]:
from collections.abc import Callable
import json
import os
from time import sleep
from typing import Any

from dadata import Dadata
from dotenv import load_dotenv
from langgraph_app.api.yazzh_final import ApiClientUnified
import rich
from ymaps import GeocodeAsync

load_dotenv()

YANDEX_API_KEY = os.getenv('YANDEX_API_KEY')
DADATA = os.getenv('DADATA')

async_yazzh_client = ApiClientUnified(verbose=True)


async def do_request(func: Callable, **kwargs) -> dict:
    async with async_yazzh_client as _client:
        result = await func(**kwargs)
        json_res = result['json']
        return json_res['data']


In [2]:
geocode_client = GeocodeAsync(YANDEX_API_KEY)
dadata = Dadata(DADATA)

In [3]:
async def coords_to_address(lat: float, lon: float):
    """
    Обратное геокодирование: по координатам (lat, lon) вернуть полный адрес.
    Возвращает строку-адрес или None, если ничего не найдено.
    """

    # Яндекс ждёт [lon, lat], то есть [долгота, широта]
    coords = [lon, lat]

    data = await geocode_client.reverse(
        coords,  # positional arg = geocode
        results=1,
        format='json',
        # при желании можно добавить kind='house' или kind='metro'
    )

    collection = data['response']['GeoObjectCollection']
    members = collection.get('featureMember', [])
    if not members:
        return None

    geo_obj = members[0]['GeoObject']
    meta = geo_obj.get('metaDataProperty', {}).get('GeocoderMetaData', {})

    # Основной адрес
    text = meta.get('text')
    if text:
        return text

    # Фоллбэк — собрать из name/description, если вдруг text отсутствует
    name = geo_obj.get('name')
    desc = geo_obj.get('description')
    if name or desc:
        return ', '.join(x for x in (name, desc) if x)

    return None


In [4]:
await coords_to_address(60.050182, 30.443045)

'Россия, Ленинградская область, Всеволожский район, Мурино, Привокзальная площадь, 6'

In [5]:
async def address_to_coord(user_address: str):
    """
    Геокодирование адреса в координаты.
    Возвращает [lat, lon] или None, если ничего не найдено.
    """
    lower_user_address = user_address.lower()
    if (
        'спб' not in lower_user_address
        and 'санкт-петербург' not in lower_user_address
        and 'санкт петербург' not in lower_user_address
    ):
        user_address = 'Санкт-Петербург, ' + user_address

    data = await geocode_client.geocode(user_address, results=1, format='json')

    collection = data['response']['GeoObjectCollection']
    members = collection.get('featureMember', [])
    if not members:
        return None

    geo_obj = members[0]['GeoObject']

    # "lon lat" (строка)
    pos_str = geo_obj['Point']['pos']
    lon_str, lat_str = pos_str.split()

    lon = float(lon_str)
    lat = float(lat_str)

    # возвращаем в привычном порядке (lat, lon)
    return [lat, lon]

In [6]:
await address_to_coord('проспект Науки, 19к1')

[60.01275, 30.395935]

In [7]:
coords = await address_to_coord('Площадь Александра Невского')
coords

[59.924068, 30.386925]

In [8]:
def suggest_spb_metro_raw(query: str) -> list[dict[str, Any]]:
    """
    Сырые подсказки по станциям метро Санкт-Петербурга.
    Просто обёртка над dadata.suggest(name="metro").
    """
    filters = [{'city': 'Санкт-Петербург'}]
    return dadata.suggest(name='metro', query=query, filters=filters)


In [9]:
def list_spb_metro_stations(include_closed: bool = False) -> list[dict[str, Any]]:
    """
    Пытается собрать список станций метро Санкт-Петербурга через DaData.

    ВАЖНО: это хак. DaData не даёт штатного метода "отдать весь справочник".
    Под капотом:
    - перебираем все биграммы русских букв ("аа", "аб", ..., "яя"),
    - вызываем dadata.suggest("metro", query=..., filters=[...]),
    - собираем и дедуплицируем станции по (name, line_name).

    Возвращает список словарей:
    {
        "metro_name": str,
        "metro_line": str | None,
        "line_id": str | None,
        "coords": {"lat": float | None, "lon": float | None},
        "city": str | None,
        "color": str | None,
        "is_closed": bool | None,
    }
    """

    # алфавит
    letters = [chr(code) for code in range(ord('а'), ord('я') + 1)]
    # можно добавить "ё"
    # letters.append('ё')

    # все биграммы
    seeds: list[str] = [a + b for a in letters for b in letters]
    base_filter: dict[str, Any] = {
        'city_kladr_id': '7800000000000',
    }
    if not include_closed:
        base_filter['is_closed'] = False

    seen: set[tuple[str, str]] = set()
    stations: list[dict[str, Any]] = []

    for q in seeds:
        suggestions = dadata.suggest(
            name='metro',
            query=q,
            count=10,  # не слишком большой, чтобы не прожигать лимиты
            filters=[base_filter],
        )

        if not suggestions:
            continue

        for s in suggestions:
            data = s.get('data', {})

            name = data.get('name')
            line_name = data.get('line_name') or ''
            if not name:
                continue

            key = (name, line_name)
            if key in seen:
                continue
            seen.add(key)

            geo_lat = data.get('geo_lat')
            geo_lon = data.get('geo_lon')

            stations.append(
                {
                    'metro_name': name,
                    'metro_line': data.get('line_name'),
                    'line_id': data.get('line_id'),
                    'coords': {
                        'lat': float(geo_lat) if geo_lat else None,
                        'lon': float(geo_lon) if geo_lon else None,
                    },
                    'city': data.get('city'),
                    'color': data.get('color'),
                    'is_closed': data.get('is_closed'),
                }
            )

    return stations


In [10]:
stations = list_spb_metro_stations(include_closed=True)
print(len(stations))
for s in stations:
    print(s['metro_line'], '-', s['metro_name'], s['coords'])

73
Кировско-Выборгская - Автово {'lat': 59.867325, 'lon': 30.261337}
Фрунзенско-Приморская - Адмиралтейская {'lat': 59.935867, 'lon': 30.31523}
Кировско-Выборгская - Академическая {'lat': 60.012806, 'lon': 30.396044}
Невско-Василеостровская - Площадь Александра Невского 1 {'lat': 59.924369, 'lon': 30.384989}
Лахтинско-Правобережная - Площадь Александра Невского 2 {'lat': 59.923563, 'lon': 30.383421}
Кировско-Выборгская - Балтийская {'lat': 59.907211, 'lon': 30.299578}
Невско-Василеостровская - Беговая {'lat': 59.98778, 'lon': 30.202501}
Лахтинско-Правобережная - Проспект Большевиков {'lat': 59.919838, 'lon': 30.466757}
Фрунзенско-Приморская - Бухарестская {'lat': 59.883769, 'lon': 30.368932}
Невско-Василеостровская - Василеостровская {'lat': 59.942577, 'lon': 30.278254}
Кировско-Выборгская - Проспект Ветеранов {'lat': 59.84211, 'lon': 30.250588}
Кировско-Выборгская - Владимирская {'lat': 59.927628, 'lon': 30.347898}
Фрунзенско-Приморская - Волковская {'lat': 59.896023, 'lon': 30.35754}

In [11]:
dadata.suggest('metro', 'Технологический')

[{'value': 'Технологический институт 1',
  'unrestricted_value': 'Технологический институт 1 (Кировско-Выборгская)',
  'data': {'city_kladr_id': '7800000000000',
   'city_fias_id': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
   'city': 'Санкт-Петербург',
   'name': 'Технологический институт 1',
   'line_id': '1',
   'line_name': 'Кировско-Выборгская',
   'geo_lat': 59.916512,
   'geo_lon': 30.318485,
   'color': 'D6083B',
   'is_closed': None}},
 {'value': 'Технологический институт 2',
  'unrestricted_value': 'Технологический институт 2 (Московско-Петроградская)',
  'data': {'city_kladr_id': '7800000000000',
   'city_fias_id': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
   'city': 'Санкт-Петербург',
   'name': 'Технологический институт 2',
   'line_id': '2',
   'line_name': 'Московско-Петроградская',
   'geo_lat': 59.916512,
   'geo_lon': 30.318485,
   'color': '0078C9',
   'is_closed': None}}]

In [12]:
SPB_METRO_STATIONS_2025_RU = [
    # Линия 1 — Кировско-Выборгская (M1)
    'Девяткино',
    'Гражданский проспект',
    'Академическая',
    'Политехническая',
    'Площадь Мужества',
    'Лесная',
    'Выборгская',
    'Площадь Ленина',
    'Чернышевская',
    'Площадь Восстания',
    'Владимирская',
    'Пушкинская',
    'Технологический институт 1',
    'Балтийская',
    'Нарвская',
    'Кировский завод',
    'Автово',
    'Ленинский проспект',
    'Проспект Ветеранов',
    # Линия 2 — Московско-Петроградская (M2)
    'Парнас',
    'Проспект Просвещения',
    'Озерки',
    'Удельная',
    'Пионерская',
    'Чёрная речка',
    'Петроградская',
    'Горьковская',
    'Невский проспект',
    'Сенная площадь',
    'Технологический институт 2',
    'Фрунзенская',
    'Московские ворота',
    'Электросила',
    'Парк Победы',
    'Московская',
    'Звёздная',
    'Купчино',
    # Линия 3 — Невско-Василеостровская (M3)
    'Беговая',
    'Зенит',
    'Приморская',
    'Василеостровская',
    'Гостиный двор',
    'Маяковская',
    'Площадь Александра Невского 1',
    'Елизаровская',
    'Ломоносовская',
    'Пролетарская',
    'Обухово',
    'Рыбацкое',
    # Линия 4 — Лахтинско-Правобережная (M4)
    'Горный институт',
    'Спасская',
    'Достоевская',
    'Лиговский проспект',
    'Площадь Александра Невского 2',
    'Новочеркасская',
    'Ладожская',
    'Проспект Большевиков',
    'Улица Дыбенко',
    # Линия 5 — Фрунзенско-Приморская (M5)
    'Комендантский проспект',
    'Старая Деревня',
    'Крестовский остров',
    'Чкаловская',
    'Спортивная',
    'Адмиралтейская',
    'Садовая',
    'Звенигородская',
    'Обводный канал',
    'Волковская',
    'Бухарестская',
    'Международная',
    'Проспект Славы',
    'Дунайская',
    'Шушары',
]


In [13]:
CITY_KLADR_SPB = '7800000000000'

In [14]:
def get_spb_metro_info(
    station_name: str,
    include_closed: bool = False,
) -> dict[str, Any] | None:
    """
    Берёт станцию метро в СПб и возвращает словарь с:
    - metro_name: название станции
    - metro_line: название линии
    - line_id: id линии (как в Dadata)
    - coords: [lat, lon]
    - city: город
    - is_closed: закрыта / нет
    - raw: сырой ответ Dadata (на всякий случай)
    """
    filters = [
        {'city_kladr_id': CITY_KLADR_SPB},
    ]
    if not include_closed:
        filters.append({'is_closed': False})

    # suggest("metro", ...) — поиск по name+city
    suggestions = dadata.suggest(
        'metro',
        station_name,
        count=5,
        filters=filters,
    )

    if not suggestions:
        return None

    # берем первый совпавший вариант
    item = suggestions[0]
    data = item['data']

    # Dadata возвращает координаты строками
    lat_str = data.get('geo_lat')
    lon_str = data.get('geo_lon')

    lat = float(lat_str) if lat_str is not None else None
    lon = float(lon_str) if lon_str is not None else None

    return {
        'metro_name': data.get('name'),
        'metro_line': data.get('line_name'),
        'line_id': data.get('line_id'),
        'coords': [lat, lon] if lat is not None and lon is not None else None,
        'city': data.get('city'),
        'is_closed': data.get('is_closed'),
        'color': data.get('color'),
        'raw': item,
    }

In [15]:
def build_spb_metro_directory(include_closed: bool = False) -> list[dict[str, Any]]:
    result: list[dict[str, Any]] = []
    for name in SPB_METRO_STATIONS_2025_RU:
        info = get_spb_metro_info(name, include_closed=include_closed)
        if info is None:
            print(f'[WARN] Не нашёл станцию в Dadata: {name}')
            continue
        result.append(info)
        # немного подтормаживаем, чтобы не долбить API
        sleep(0.1)
    return result


def dump_spb_metro_directory(path: str = 'spb_metro_dadata.json') -> None:
    data = build_spb_metro_directory(include_closed=False)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f'Справочник записан в {path}')

In [16]:
async def add_addresses_to_metro_items(items: list[dict[str, Any]]) -> list[dict[str, Any]]:
    """
    Проходит по списку станций метро, для каждой coords = [lat, lon]
    добавляет поле 'address' через coords_to_address.
    """
    result: list[dict[str, Any]] = []

    for item in items:
        coords = item.get('coords')

        # если координат нет или кривые — просто добавляем address=None
        if not coords or len(coords) != 2 or coords[0] is None or coords[1] is None:
            new_item = dict(item)
            new_item['address'] = None
            result.append(new_item)
            continue

        lat, lon = coords  # из файла [lat, lon]
        addr: str | None = await coords_to_address(lat, lon)

        new_item = dict(item)
        new_item['address'] = addr
        result.append(new_item)

    return result


async def enrich_metro_file_with_addresses(
    input_path: str,
    output_path: str,
) -> None:
    """
    Читает JSON со списком станций метро, добавляет поле 'address'
    через Яндекс-геокодер и сохраняет в новый файл.
    """
    with open(input_path, encoding='utf-8') as f:
        data = json.load(f)

    if not isinstance(data, list):
        raise ValueError('Ожидался JSON-массив (список станций), а не объект')

    enriched = await add_addresses_to_metro_items(data)

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(enriched, f, ensure_ascii=False, indent=2)

    print(f'Готово: записано в {output_path}')


In [17]:
await enrich_metro_file_with_addresses(
    'spb_metro_dadata.json',
    'spb_metro_dadata_with_address.json',
)

Готово: записано в spb_metro_dadata_with_address.json
